<center>

<h1 style="text-align:center"> Modular Programming </h1>
</center>

## Review

* Previously
  + How to build small programs
  
* This lecture
  + How to build at scale: Structures, Signatures, Functors.
  
$
\require{color}
\newcommand{\cred}[1]{{\color{\red}{\text{#1}}}}
\newcommand{\cgreen}[1]{{\color{\green}{\text{#1}}}}
$

## Scale

* **Assignment 1 & 2** : ~100 lines of code
* **OCaml** : 375000 lines of code
* **Hubble space telescope** : 2 million lines of code
* **Facebook** : 60 million lines of code (estimated)
* **Google (all services)** : 2 billion lines of code (estimated)

...can’t be done by one person

...no individual programmer can understand all the details

...$\cred{too complex to build with OCaml we’ve seen so far}$

## Modularity

* Code comprises of individual **Modules**
* Developed separately
  + Reason $\cgreen{locally}$, not $\cred{globally}$.
* Clearly specified **Interfaces** for using the modules

<center>

<img src="modules.png" width=250>
</center>

## Features of modularity

* **Namespacing**
  + Provide way to name a collection of related features (`List`, `Set`)
  + Avoid name clashes with similarly named features from different collections (`insert` in `List` and `Set`)

## Features of modularity

* **Abstraction**
  + Hide the details of implementation
  + Avoid the user breaking invariants implicit in the code.
    + Example: LIFO property while implementing a stack using `List`. 
    + User sorts the list which will break the LIFO property.
  + Transparently change the implementation without breaking the code of the client (i.e. user) of the module.
    + Implement stack with an `Array` rather than a `List`.

## Features of modularity

* **Code reuse**
  + Avoid reimplementing features that are already present.
  + Implementing stack using `List` : Reuse `List` functions such as `is_empty` and `length`.

## OCaml Features for Modularity

* **Namespacing:** Structure
* **Abstraction:** Signature
* **Code Reuse:** Functors, includes, and sharing constraints.

## Structure

* A collection of **definitions**.
* Evaluated in order
* Structure value can be bound to a name

The syntax is 

```ocaml
module Module_name = struct 
  (* collection of definitions *) 
end
```

## Structure

+ Let us implement a purely functional stack using OCaml's built-in list data structure.
+ In a purely functional data structure, "update" operations return a new version of the data structure.
  + In a stack, push and pop return a new version of the stack.

In [ ]:
module Stack = struct
  let empty = []
  let push v s = v::s
  let pop s = match s with
  | [] -> None
  | x::xs -> Some (x, xs)
  let depth s = List.length s
end

## Structures provide namespacing

In [ ]:
Stack.empty

In [ ]:
empty

## Signature

* A collection of **declarations**.
* No evaluation, only used for type-checking. 
* Signature type can also be bound to a name. 

```ocaml
module type Module_type_name = sig
  (* collection of declaration *)
end
```

## Stack Signature

In [ ]:
module type StackType = sig
  val empty : 'a list
  val push : 'a -> 'a list -> 'a list
  val pop : 'a list -> ('a * 'a list) option
  val depth : 'a list -> int
end

## Handy Jupyter Functions 

Jupyter (really the OCaml top-level) has the following handy functions to display the modules and module signatures.

In [ ]:
#show_module Stack

In [ ]:
#show_module_type StackType

## Explicit Signatures

In [ ]:
module M : StackType = Stack

## Hiding functionality with explicit signatures

In [ ]:
module M : (sig 
  val empty : 'a list 
end) = Stack

In [ ]:
Stack.empty;;
Stack.push 1 Stack.empty

In [ ]:
M.empty;;
M.push 1 M.empty

## Opening modules

* Use `open` to make available all the definitions from the module

In [ ]:
open Stack

In [ ]:
empty

## Including module functionality

* `include` allows new modules to be constructed by extending earlier modules (Code reuse).

In [ ]:
module Stack = struct
  include Stack
  let is_empty s = match s with
  | [] -> true
  | _ -> false
end

## Include on signatures

`include` also works on signatures to define other signatures.

In [ ]:
module type MT = sig
  include StackType
  val is_empty : 'a list -> bool
end

## Abstract types

* So far we have only seen how to collect definitions under a common name (Namespacing)
  + The implementation details are still visible.

In [ ]:
let s = Stack.empty |> Stack.push 1 |> Stack.push 2 |> Stack.push 3

Aside: `|>` is called the pipelining operator in OCaml. The above is equivalent to:

In [ ]:
let s1 = Stack.push 3 (Stack.push 2 (Stack.push 1 Stack.empty))

## Abstract types


`s` is of type `int list`

Can do non-sensical operations for a stack such as `List.sort`.

In [ ]:
List.sort (-) s

## Abstract stack

**Abstract** the type of stack such that only those operations allowed by the interface are applicable. 

Define the stack signature with abstract stack type.

In [ ]:
module type AbsStackType = sig
  type 'a t (* abstract type *)
  val empty    : 'a t
  val push     : 'a -> 'a t -> 'a t
  val pop      : 'a t -> ('a * 'a t) option
  val depth    : 'a t -> int
  val is_empty : 'a t -> bool
end

## Abstract stack

In [ ]:
module AbsStack : AbsStackType = struct
  type 'a t = 'a list
  include Stack
end

In [ ]:
let s = AbsStack.empty |> AbsStack.push 1 |> AbsStack.push 2 |> AbsStack.push 3;;

In [ ]:
AbsStack.pop s

In [ ]:
List.sort (-) s

## Abstraction

* Interfaces with abstract types are **contracts** between the *implementer* and the *user*.
* User can interact with the values of the type only using the API
  + No way to sort the internal list anymore.
* Implementer can transparently change the internals of the implementation 
  + Implement the stack with an array rather than list
  + As long as the same API is preserved, the user code does not break.

## Hiding internals using module types

In [ ]:
module M1 = struct
  type t = int
  let v : t = 5
end;;

M1.v

In [ ]:
module type T = sig
  type t
  val v : t
end;;

module M2: T = struct type t = int let v : t = 5 end;;

M2.v

## Stack using user-defined list

In [ ]:
module AbsVariantStack : AbsStackType = struct
  type 'a t = 
  | Nil 
  | Cons of 'a * 'a t
  
  let empty = Nil
  
  let rec depth_aux acc l = match l with
  | Nil -> acc
  | Cons (x, xs) -> depth_aux (1+acc) xs
  
  let depth l = depth_aux 0 l
  
  let push v s = Cons (v,s)
  
  let pop s = match s with Nil -> None | Cons (x,xs) -> Some (x,xs)
  
  let is_empty s = match s with Nil -> true | _ -> false
end

## Two Stack Modules

In [ ]:
#show_module AbsStack;;
#show_module AbsVariantStack;;
#show_module_type AbsStackType;;


Wherever the user uses the `AbsStack`, we can replace that with `AbsVariantStack` and the user code wouldn't be able to tell the difference. 

## Code reuse: Module functors


* We want to create new modules using existing modules
  + One possibility is to use `include`, but this just copies the existing module.
* Module functor: a function which takes as input module(s) and generates a new module

## Identity functor

The simplest function is the identity function

In [ ]:
let id (x : int) : int = x

At the module level, we can correspondingly define the identify functor

In [ ]:
module type T = sig 
  type t 
  val v : t 
end

In [ ]:
module Id (X : T) : T = X

## Applying Identity functor

In [ ]:
id 5

Similarly:

In [ ]:
module M = struct 
  type t = int
  let v = 10
end

In [ ]:
module M' = Id(M)

## Type equality under abstraction

* We know `M` and `M'` are the same modules.
  + Let's ask whether `M.v = M'.v`.

In [ ]:
M.v = M'.v

* While the compiler knows that the type `M.t` is `int`, the type `M'.t` (return type of functor) is **abstract**
  + Compiler does not know that `M.t` is the same type as `M'.t`. 

## Sharing constraints

* Use **sharing constraints** to let the compiler know more about the abstract types
* Sharing constraints make the module types _less abstract / more concrete_. 

Consider the original identity functor definition

In [ ]:
module Id (X : T) : T = X

Now consider the identity functor with sharing constraint on the return type `T`

In [ ]:
module Id2 (X : T) : (T with type t = X.t)
  = X

## Sharing Constraints

In [ ]:
module M2 = Id2(M)

In [ ]:
M2.v = M.v

## Multiple sharing constraints

In [ ]:
module type T2 = sig 
  type t
  type u 
end

module Id2 (X2 : T2) : 
  (T2 with type t = X2.t
       and type u = X2.u) = struct 
  type t = X2.t
  type u = X2.u
end

## Functor Example: Serializable List

* Write a `string_of_list` and `list_of_string` functions for _any typed_ list.
    + We need to know how to convert the list element to string and vice-versa.
* For an `int list`, we can write:

In [ ]:
let string_of_list l = 
  let rec loop acc l = match l with
    | [] -> acc
    | [x] -> acc ^ (string_of_int x)
    | x::xs -> loop (acc ^ (string_of_int x) ^ ";") xs
  in
  "[" ^ (loop "" l) ^ "]"  

In [ ]:
string_of_list [1;2;3]

## Functor Example: Serializable List

In [ ]:
let list_of_string s =
  let s = String.sub s 1 (String.length s - 2) in
  List.map int_of_string (String.split_on_char ';' s)

In [ ]:
list_of_string "[1;2;3]"

## Generalise

What about float, tuples, records, etc? Use **Functors**.

In [ ]:
module type Serializable = sig
  type t
  val t_of_string : string -> t
  val string_of_t : t -> string
end

## Generalise

What about float, tuples, records, etc? Use **Functors**.

In [ ]:
module SerializableList (C : Serializable) : Serializable = struct
  type t = C.t list
  
  let string_of_t l = 
    let rec loop acc l = match l with
      | [] -> acc
      | [x] -> acc ^ (C.string_of_t x)
      | x::xs -> loop (acc ^ (C.string_of_t x) ^ ";") xs
    in
    "[" ^ (loop "" l) ^ "]"  
    
  let t_of_string s =
    let s = String.sub s 1 (String.length s - 2) in
    List.map C.t_of_string (String.split_on_char ';' s)
end

## SerializableFloatList

In [ ]:
module SerializableFloatList = SerializableList (struct 
  type t = float
  let t_of_string = float_of_string
  let string_of_t = string_of_float
end)

## SerializableFloatList

In [ ]:
SerializableFloatList.string_of_t [1.4;2.3;3.4]

In [ ]:
SerializableFloatList.t_of_string "[1.4;2.3;3.4]"

## SerializableStringList

In [ ]:
module SerializableStringList = SerializableList (struct 
  type t = string
  let t_of_string x = x
  let string_of_t x = x
end)

## SerializableStringList

In [ ]:
SerializableStringList.string_of_t ["Et";"tu,";"Brute?"]

In [ ]:
SerializableStringList.t_of_string "[Et;tu,;Brute?]"

## Behold the power of abstraction

Observe that the signature of `SerializableFloatList` is also `Serializable`.

In [ ]:
module SerializableFloatListList = SerializableList (SerializableFloatList)

In [ ]:
SerializableFloatListList.string_of_t [[1.1]; [2.1;2.2]; [3.1;3.2;3.3]]

## Functors $\neq$ Functions


* OCaml is a stratified language.
  + Values + Expressions and Types at term level.
  + Structures and Signatures at module level.
  + In particular, a module is not a value.
* Hence, functors are not equivalent to functions
  + They are not "first-class".
  + You cannot pass a functor to another functor.